Copyright 2024 Lehi Gracia

In [3]:
STOCK = 'NVDA'

In [4]:

import warnings
warnings.filterwarnings('ignore')

import yfinance
from datetime import datetime

t = yfinance.Ticker(STOCK)
n = t.news
for i in n:
    print(f"{datetime.fromtimestamp(i['providerPublishTime']).strftime('%Y-%m-%d')}  {i['title']}-Lehi")

ModuleNotFoundError: No module named 'yfinance'

### Ratings UP and DOWN

In [ ]:

from yfinance_utils import financials_utils

ratings = financials_utils.get_ratings(t)
for i in ratings.keys():
    print(f"{ratings[i]}  {i}")



### Total Ratings

In [ ]:

import plotly.express as px
from collections import Counter

dupdown = t.get_upgrades_downgrades()
grades = list(dupdown['ToGrade'])

values = list(Counter(grades).values())
keys = list(Counter(grades).keys())

fig3 = px.pie(values=values, names=keys, title=f'Ratings for {STOCK}', width=600, height=400)
fig3.update_layout()

fig3.show()

In [ ]:

from finta import TA

data = t.history(period='1y', rounding=True)
data['ema10'] = TA.EMA(data, 10)
data['sma30'] = TA.SMA(data, 30)

import plotly.graph_objects as go

# Stock and moving averages
fig = go.Figure(data=[
                go.Candlestick(
                    x = data.index, # date values
                    open  = data['Open'],
                    high  = data['High'],
                    low   = data['Low'],
                    close = data['Close'],
                    name = STOCK
                ),
         
                go.Scatter( x=data['ema10'].index,
                            y=data['ema10'],
                            name='EMA 10',
                            line=dict(color='blue',
                                      width=2,
                                      shape='spline'), # smooth the line
                            ),
        
                go.Scatter( x=data['sma30'].index,
                            y=data['sma30'],
                            name='SMA 30',
                            line=dict(color='purple',
                                      width=2,
                                      shape='spline'), # smooth the line
                            ),
                
                ])
fig.update_layout(xaxis_rangeslider_visible=False, title=f"{STOCK}", width=600, height=400)
fig.show()

# Volume
data['vol_avg'] = data['Volume'].rolling(14).mean()
fig1 = go.Figure(
    data = [
        go.Bar(x=data.index, y=data['Volume'], name='Volume'),
        go.Scatter(x=data.index, y=data['vol_avg'], name='Average')
    ])

fig1.update_layout(title=f"{STOCK}", width=600, height=400)
fig1.show()


In [ ]:

from yfinance_utils import signals_utils
#
#Red = '\033[91m'
#Green = '\033[92m'
#Blue = '\033[94m'
#Cyan = '\033[96m'
#White = '\033[97m'
#Yellow = '\033[93m'
#Magenta = '\033[95m'
#Grey = '\033[90m'
#Black = '\033[90m'
#Default = '\033[99m'

def get_color_dot(c):
    if c:
        return"\033[92m ● "  # Green dot
    else:
        return"\033[90m ● "  # Grey dot

print(f"STOCK:  {STOCK}")
print(f"{get_color_dot(signals_utils.is_vix_hma_cross_up(day=2))} VIX has crossed up")
print(f"{get_color_dot(signals_utils.is_vix_hma_cross_up(day=2))} VIX has crossed down")
print(f"{get_color_dot(signals_utils.is_macd_cross_up(data))} MACD has crossed up")
print(f"{get_color_dot(signals_utils.is_macd_cross_down(data))} MACD has crossed down")
print(f"{get_color_dot(signals_utils.is_stochastic_cross_up(data))} Stochastic has crossed up")
print(f"{get_color_dot(signals_utils.is_stochastic_cross_down(data))} Stochastic has crossed down")
print(f"{get_color_dot(signals_utils.is_ma_bullish_trend(data))} Moving average is bullish")
print(f"{get_color_dot(signals_utils.is_ma_trend_reversing(data))} Moving average is reversing")
print(f"{get_color_dot(signals_utils.is_ma_cross_down(data))} Moving average cross down")
print(f"{get_color_dot(signals_utils.is_ma_cross_up(data))} Moving average cross up")
print(f"{get_color_dot(signals_utils.is_ma_bearish_trend(data))} Moving average is bearish")
print(f"{get_color_dot(signals_utils.is_ma_price_cross_down(data))} Moving average price crossed below ")
print(f"{get_color_dot(signals_utils.is_ma_price_cross_up(data))} Moving average price crossed above ")
print(f"{get_color_dot(signals_utils.is_price_year_low(data))} 52 week low ")
print(f"{get_color_dot(signals_utils.is_price_year_high(data))} 52 week high ")
print(f"{get_color_dot(signals_utils.is_rsi_oversold(data))} RSI oversold (30)")
print(f"{get_color_dot(signals_utils.is_rsi_overbough(data))} RSI overbought (70)")
print(f"{get_color_dot(signals_utils.is_price_year_low(data))} RSI 52 week low")
print(f"{get_color_dot(signals_utils.is_price_year_high(data))} RSI 52 week high")
print(f"{get_color_dot(signals_utils.is_resistance_breake_up(data))} Resistance break up")
print(f"{get_color_dot(signals_utils.is_support_breake_down(data))} Support break down")
print(f"{get_color_dot(signals_utils.is_cahold(data))} Close Above High Of Low Day")
print(f"{get_color_dot(signals_utils.is_cblohd(data))} Close Below Low Of High Day")
print(f"{get_color_dot(signals_utils.is_high_volume(data))} Volume high")


In [ ]:

from yfinance_utils import ratio_utils
import pandas as pd

print(f"Stock                        : {STOCK}")
print(f"Price                        : {t.history()['Close'].iloc[-1].round(2)}")
print(f"Put Call Ratio               : {ratio_utils.get_put_call_ratio(t):,.2f}")
print(f"Current Ratio                : {ratio_utils.get_current_ratio(t):,.2f}")
print(f"Book Value per Share         : {ratio_utils.get_book_value_per_share_ratio(t):,.2f}")
print("")

df = pd.DataFrame()

ls_name = [
    'Revenue MM'                  ,
    'Average Assets MM'           ,
    'Average Inventory MM'        ,
    'Revenue per Share'           ,
    'Earnings per Share'          ,
    'Share Price to Earnings'     ,
    'Price to Sales'              ,
    'Acid Test Ratio'             ,
    'Cash Ratio'                  ,
    'Operating Cash Flow Ratio'   ,
    'Debt Ratio'                  ,
    'Debt to Equity Ratio'        ,
    'Interest Coverage Ratio'     ,
    'Service Coverage Ratio'      ,
    'Asset Turnover Ratio'        ,
    'Inventory Turnover Ratio'    ,
    'Day Sales in Inventory Ratio',
    'Return on Assets Ratio'      , 
    'Return on Equity Ratio'
]

for i in range(1,3):
    name = ls_name
    
df['NAME'] = ls_name
df.set_index('NAME',inplace=True)

for i in range(2):
    ls = []
    ls.append(ratio_utils.get_net_revenue(t, period = i)/1000000)
    ls.append(ratio_utils.get_avg_assets(t, period = i)/1000000)
    ls.append(ratio_utils.get_avg_inventory(t, period = i)/1000000)
    ls.append(ratio_utils.get_revenue_per_share(t, period = i))
    ls.append(ratio_utils.get_earnings_per_share(t, period = i))
    ls.append(ratio_utils.get_price_to_earnings(t, period = i))
    ls.append(ratio_utils.get_price_to_sales(t, period = i))
    ls.append(ratio_utils.get_acid_test_ratio(t, period = i))
    ls.append(ratio_utils.get_cash_ratio(t, period = i))
    ls.append(ratio_utils.get_operating_cash_flow_ratio(t, period = i))
    ls.append(ratio_utils.get_debt_ratio(t, period = i))
    ls.append(ratio_utils.get_debt_to_equity_ratio(t, period = i))
    ls.append(ratio_utils.get_interest_coverage_ratio(t, period = i))
    ls.append(ratio_utils.get_service_coverage_ratio(t, period = i))
    ls.append(ratio_utils.get_asset_turnover_ratio(t, period = i))
    ls.append(ratio_utils.get_inventory_turnover_ratio(t, period = i))
    ls.append(ratio_utils.get_day_sales_in_inventory_ratio(t, period = i))
    ls.append(ratio_utils.get_return_on_assets_ratio(t, period = i))
    ls.append(ratio_utils.get_return_on_equity(t, period = i))
    df[f"PERIOD_{i}"] = ls

direction = []
for i in range(len(df)):
    if df['PERIOD_0'].iloc[i] > df['PERIOD_1'].iloc[i]:
        pct = round(((df['PERIOD_0'].iloc[i] / df['PERIOD_1'].iloc[i]) - 1) * 100,0)
        direction.append(f"\033[92m UP {abs(pct)}% ● \033[97m")
    else:
        pct = round(((1-((df['PERIOD_0'].iloc[i] / df['PERIOD_1'].iloc[i])))*100),0)
        direction.append(f"\033[91m DOWN {abs(pct)}% ● \033[97m")
df['CHANGE'] = direction

print("Records for last two periods.")
print("Note: periods are read right to left")
print('')
print(df.round(2))